In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_24weeks_hf_male = sc.read_h5ad('../../exported_data/pvat_24weeks_hf_male.h5ad')
pvat_24weeks_hf_male

AnnData object with n_obs × n_vars = 13385 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered',

In [3]:
pvat_24weeks_hf_male.obs['celltype_broad'].unique().tolist()

['Adipocytes',
 'Endothelial_Cells',
 'Immune_Cells',
 'Fibroblasts',
 'Mesothelial_Cells',
 'Pericytes',
 'SMCs',
 'Neuronal_Cells']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_24weeks_hf_male.raw = pvat_24weeks_hf_male
pvat_24weeks_hf_male.raw.X

<13385x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 20296333 stored elements in Compressed Sparse Row format>

In [6]:
# Run CellPhoneDB analysis on PVAT data
cellphonedb(pvat_24weeks_hf_male, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellphonedb_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
966 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.18 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 13385 samples and 19777 features


100%|███████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 168.70it/s]


In [7]:
ligand_receptor_pvat = pvat_24weeks_hf_male.uns['cellphonedb_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
2634,Slit3,Slit3,30.021278,0.957447,Robo4,Robo4,0.391951,0.281211,SMCs,Endothelial_Cells,15.206614,0.0
9140,Slit3,Slit3,30.021278,0.957447,Robo2,Robo2,0.250000,0.250000,SMCs,Neuronal_Cells,15.135639,0.0
10501,Sorbs1,Sorbs1,14.349764,0.979381,Itga1,Itga1,4.063830,0.808511,Adipocytes,SMCs,9.206797,0.0
11726,Ncam1,Ncam1,2.375000,0.875000,Cacna1c,Cacna1c,14.659573,0.957447,Neuronal_Cells,SMCs,8.517286,0.0
46,Ntrk3,Ntrk3,14.862913,0.943411,Ptprs,Ptprs,1.675024,0.686015,Adipocytes,Adipocytes,8.268969,0.0


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellphone_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
270,Col4a1,Col4a1,3.926878,0.652688,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,2.193492,0.0
307,Col3a1,Col3a1,3.433331,0.750538,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.946718,0.0
276,Col4a2,Col4a2,3.183868,0.609677,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.821987,0.0
359,Col4a2,Col4a2,3.183868,0.609677,Itgb5,Itgb5,0.431755,0.273196,Fibroblasts,Adipocytes,1.807811,0.0
301,Col5a3,Col5a3,2.905374,0.688172,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.682739,0.0
264,Col6a6,Col6a6,2.313977,0.526882,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.387041,0.0
303,Col5a2,Col5a2,2.092471,0.669892,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.276288,0.0
280,Col6a2,Col6a2,1.833332,0.656989,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.146718,0.0
282,Col6a3,Col6a3,1.645160,0.559140,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,1.052633,0.0
305,Col5a1,Col5a1,1.323655,0.637634,Itgb1,Itga1_Itgb1,0.460105,0.304684,Fibroblasts,Adipocytes,0.891880,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_24weeks_hf_male_cellphonedb.csv")
pvat_24weeks_hf_male.write("../../exported_data/pvat_24weeks_hf_male.h5ad")